# pymofa tutorial
last updated: 2016-09-02

This notebook introduces the basic functionalities of pymofa, the python modeling framework to run and evaluate your models systematically ;)

In [4]:
cd ..

/home/barfuss/ZimBox/Software


... to pymofa root

In [5]:
from pymofa import experiment_handling as eh

## dummy model
First we need to create a dummy model.

In [6]:
def dummy_model(mu, std, time_length):
    """Defining the dummy model."""